In [30]:
import os
from openai import OpenAI
import json


api_key = "sk-proj-TTKUOiShoeid4amkQXnkT3BlbkFJuNv30RXFZduPHpghIKAG"

client = OpenAI(api_key=api_key)

OPENAI_API_KEY = api_key

In [31]:
from langchain_community.document_loaders import DirectoryLoader

DATA_PATH = "../kurs1/texts/"

def load_documnets():
    loader = DirectoryLoader(DATA_PATH,glob="*.txt")
    documnets = loader.load()
    return documnets


In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from typing import List


def split_text(documents: List[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks

In [33]:
from langchain_community.vectorstores import Chroma



In [43]:
CHROMA_PATH = os.path.join(os.getcwd(), "chroma_db")


In [44]:
CHROMA_PATH

'/home/dell/Desktop/my_folder/rag1/chroma_db'

In [48]:
import shutil
from langchain.embeddings import OpenAIEmbeddings


def ensure_writable_directory(path):
    """Ensure the directory exists and is writable."""
    if not os.path.exists(path):
        os.makedirs(path)
    os.chmod(path, 0o777)  # Ensure the directory has write permissions

# Step 2: Save the chunks to the Chroma database
def save_to_chroma(chunks: List[Document]):

    # ensure_writable_directory(CHROMA_PATH)
    # # Clear out the database first.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)
        ensure_writable_directory(CHROMA_PATH)  # Recreate with proper permissions

    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, OpenAIEmbeddings(api_key=api_key), persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

# Step 3: Query the Chroma database
def query_chroma(query: str):
    db = Chroma(persist_directory=CHROMA_PATH)
    results = db.query(query, OpenAIEmbeddings(api_key=api_key))
    for result in results:
        print(result['content'])
        print(result['metadata'])

In [ ]:

import chromadb

chroma_client = chromadb.Client()
# switch `create_collection` to `get_or_create_collection` to avoid creating a new collection every time
collection = chroma_client.get_or_create_collection(name="my_collection")
# switch `add` to `upsert` to avoid adding the same documents every time
collection.upsert(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)
results = collection.query(
    query_texts=["This is a query document about florida"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
print(results)

In [ ]:
# __import__('pysqlite3')
# import pysqlite3
# sys.modules['sqlite3'] = sys.modules["pysqlite3"]

In [37]:
from langchain_community.vectorstores import Chroma


In [ ]:
documents = load_documnets()
chunks = split_text(documents)

In [49]:
import time

def x(chunks):
    collection_name="my"
    recreate_db = False
    persist_directory = "./chroma_db"
    t1_start = time.perf_counter()
    if recreate_db:
        vectorstore = Chroma.from_documents(
            collection_name=collection_name, documents=chunks, embedding= OpenAIEmbeddings(api_key=api_key), persist_directory=persist_directory)
        vectorstore.persist()
    else:
        vectorstore = Chroma(collection_name=collection_name, persist_directory=persist_directory, embedding_function= OpenAIEmbeddings(api_key=api_key))
    t1_stop = time.perf_counter()  
    print("elapsed time:", t1_stop-t1_start)

In [50]:
x(chunks)

ValueError: Could not connect to tenant default_tenant. Are you sure it exists?

In [52]:

save_to_chroma(chunks)


OperationalError: attempt to write a readonly database

In [ ]:

# Query the database
query = "your search query"
query_chroma(query)

In [55]:
import logging
import os

import chromadb
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma



DB_DIR = os.path.join(os.getcwd(), "db")

def load_documnets():
    loader = DirectoryLoader(DATA_PATH,glob="*.txt")
    documnets = loader.load()
    return documnets


def init_chromadb():
    client_settings = chromadb.config.Settings(
        chroma_db_impl="duckdb+parquet",
        persist_directory=DB_DIR,
        anonymized_telemetry=False
    )
    embeddings = OpenAIEmbeddings()

    vectorstore = Chroma(
        collection_name="langchain_store",
        embedding_function=embeddings,
        client_settings=client_settings,
        persist_directory=DB_DIR,
    )

    vectorstore.add_documents(documents=load_documnets(), embedding=embeddings)
    vectorstore.persist()


def query_chromadb():
    client_settings = chromadb.config.Settings(
        chroma_db_impl="duckdb+parquet",
        persist_directory=DB_DIR,
        anonymized_telemetry=False
    )

    embeddings = OpenAIEmbeddings(api_key=api_key)

    vectorstore = Chroma(
        collection_name="langchain_store",
        embedding_function=embeddings,
        client_settings=client_settings,
        persist_directory=DB_DIR,
    )
    result = vectorstore.similarity_search_with_score(query="FREDERICK", k=4)
    print(result)


def main():
    # init_chromadb()
    query_chromadb()


if __name__ == '__main__':
    main()

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/dell/miniconda3/envs/env310/lib/python3.10/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/dell/miniconda3/envs/env310/lib/python3.10/site-packages/certifi/cacert.pem'


ValueError: An instance of Chroma already exists for ephemeral with different settings